In [2]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import log_loss
import tensorflow as tf
from tensorflow.contrib.learn import DNNClassifier
import datetime

In [3]:
gender_age_train = pd.read_csv('./input/gender_age_train.csv', index_col='device_id')

nparray = np.load('intermediates/brand_train.npz')
brand_train = csr_matrix((\
                          nparray['data'], \
                          nparray['indices'], \
                          nparray['indptr']),\
                         shape=(gender_age_train.shape[0],131))

nparray = np.load('intermediates/device_train.npz')
device_train = csr_matrix((\
                          nparray['data'], \
                          nparray['indices'], \
                          nparray['indptr']),\
                         shape=(gender_age_train.shape[0],1599))

nparray = np.load('intermediates/apps_train.npz')
app_train = csr_matrix((\
                          nparray['data'], \
                          nparray['indices'], \
                          nparray['indptr']),\
                         shape=(gender_age_train.shape[0],19237))

nparray = np.load('intermediates/label_train.npz')
label_train = csr_matrix((\
                          nparray['data'], \
                          nparray['indices'], \
                          nparray['indptr']),\
                         shape=(gender_age_train.shape[0],492))

In [4]:
Xtrain = hstack((brand_train, device_train, app_train, label_train), format='csr')
Xtrain.shape

(74645, 21459)

In [5]:
gender_age_train.head()

,gender,age,group
device_id,,,
-8076087639492063270,M,35,M32-38
-2897161552818060146,M,35,M32-38
-8260683887967679142,M,35,M32-38
-4938849341048082022,M,30,M29-31
245133531816851882,M,30,M29-31


In [6]:
group_onehot = pd.get_dummies(gender_age_train.group)

In [7]:
current_index = 0;
def get_next_batch(batch_size):
    global current_index
    train_data = Xtrain[current_index:current_index + batch_size,:]
    train_target = group_onehot[current_index:current_index + batch_size]
    current_index += batch_size
    return (train_data.todense(), train_target)

In [8]:
sess = tf.InteractiveSession()

batch_size = 100

# building model:
x = tf.placeholder(tf.float32, shape=[batch_size, 21459])
labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))
w1 = tf.Variable(tf.zeros([21459, 10000]))
b1 = tf.Variable(tf.zeros([10000]))
x1 = tf.nn.relu(tf.matmul(x, w1) + b1)

w2 = tf.Variable(tf.zeros([10000, 5000]))
b2 = tf.Variable(tf.zeros([5000]))
x2 = tf.nn.relu(tf.matmul(x1, w2) + b2)

w3 = tf.Variable(tf.zeros([5000, 12]))
b3 = tf.Variable(tf.zeros([12]))
y_predict = tf.matmul(x2,w3) + b3

sess.run(tf.initialize_all_variables())

In [9]:
loss = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits( \
        y_predict, labels_placeholder))
train_op = tf.train \
    .GradientDescentOptimizer(0.5).minimize(loss)

ValueError: Shapes (?, 12) and (?,) are not compatible

In [ ]:
for i in range(100):
    batch = get_next_batch(100)
    _, loss_value = sess.run([train_op, loss],
                               feed_dict={x: batch[0], y_: batch[1]})
    print('batch ', i , ' loss', loss_value)